In [1]:
from model import Model
import json
import os

/home/j1choi/Downloads/pyssem-main/pyssem/utils/simulation/scen_properties.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
# Load simulation configuration
with open('./example-purdue.json') as f:
  simulation_data = json.load(f)

In [3]:
scenario_props = simulation_data['scenario_properties']

In [4]:
scenario_props

{'start_date': '01/03/2024',
 'simulation_duration': 100,
 'steps': 100000,
 'min_altitude': 515,
 'max_altitude': 900,
 'v_shells': 680000,
 'launch_function': 'Lambda',
 'integrator': 'BDF',
 'density_model': 'static_exp_dens_func',
 'LC': 0.1,
 'v_imp': 10.0,
 'fragment_spreading': False,
 'parallel_processing': False,
 'basline': False}

In [5]:
# Create an instance of the Model with the simulation parameters
model = Model(
    start_date=scenario_props["start_date"].split("T")[0],  # Assuming date is in ISO format
    simulation_duration=scenario_props["simulation_duration"],
    steps=scenario_props["steps"],
    min_altitude=scenario_props["min_altitude"],
    max_altitude=scenario_props["max_altitude"],
    v_shells=scenario_props["v_shells"],
    launch_function=scenario_props["launch_function"],
    integrator=scenario_props["integrator"],
    density_model=scenario_props["density_model"],
    LC=scenario_props["LC"],
    v_imp=scenario_props["v_imp"],
    fragment_spreading = scenario_props["fragment_spreading"]
)

SHELL COUNT:  10
R0_km [515.         515.20394479 515.40772825 515.61135068 515.81481242
 516.01811377 516.22125506 516.42423659 516.62705868 516.82972165
 517.0322258 ]


In [6]:
species = simulation_data["species"]
species_list = model.configure_species(species)

Splitting species N into 2 species with masses [0.64, 223].
Added 1 active species, 2 debris species, and 0 rocket body species to the simulation.
Pairing the following active species to debris classes for PMD modeling...
['S']
Matched species S to debris species N_223kg.
    Name: N_0.64kg
    pmd_linked_species: []
    Name: N_223kg
    pmd_linked_species: ['S']


Creating collision pairs:  17%|███▏               | 1/6 [00:00<00:00,  8.99it/s]

Species pairing for  S N_0.64kg and sk  ['S', 'N_0.64kg', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'N_0.64kg'}
Species pairing for  S N_223kg and sk  ['S', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'N_223kg'}
Species pairing for  N_0.64kg N_223kg and sk  ['N_0.64kg', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'N_0.64kg'} {'N_223kg'}


Creating collision pairs:  50%|█████████▌         | 3/6 [00:00<00:00, 10.85it/s]

Species pairing for  S S and sk  ['S', 'S', 'N_0.64kg', 'N_223kg']
pairing {'S'} {'S'}
Species pairing for  N_0.64kg N_0.64kg and sk  ['N_0.64kg', 'N_0.64kg', 'N_0.64kg', 'N_223kg']
pairing {'N_0.64kg'} {'N_0.64kg'}


Creating collision pairs: 100%|███████████████████| 6/6 [00:00<00:00, 12.18it/s]

Species pairing for  N_223kg N_223kg and sk  ['N_223kg', 'N_223kg', 'N_0.64kg', 'N_223kg']
pairing {'N_223kg'} {'N_223kg'}


In [7]:
# Run the model
# ODE solution
results = model.run_model()

baseline
PER YEAR 31557600.0 1000.0
PER YEAR 31557600.0 1000.0
PER YEAR 31557600.0 1000.0
Conversion of equations to lambda functions...
x0           S  N_0.64kg   N_223kg
0  2.807405  0.373474   0.18716
1  2.811269  0.373559  0.187418
2   2.81513  0.373644  0.187675
3  2.818988  0.373728  0.187933
4  2.822841  0.373811  0.188189
5  2.826691  0.373894  0.188446
6  2.830538  0.373977  0.188703
7  2.834381  0.374058  0.188959
8   2.83822   0.37414  0.189215
9  2.842055   0.37422   0.18947
Integrating equations...
Model run completed successfully.


In [1]:
#model.create_plots()

In [ ]:
from datetime import datetime
emresults_list = []

# run iterations on EM simulations 
for i in range(0, 30):
    current_time = datetime.now()
    print(f"Iteration {i} start time: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    emresults = model.scenario_properties.run_euler_maruyama()
    emresults_list.append(emresults)

Iteration 0 start time: 2025-09-27 09:48:40


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

scenario_properties = model.scenario_properties
species_names = scenario_properties.species_names
num_shells = scenario_properties.n_shells
n_species = len(species_names)

# Calculate average and std from results list
results_array = np.array(emresults_list)  # shape: (num_runs, n_species*num_shells, n_steps)
mean_results = np.mean(results_array, axis=0)
std_results = np.std(results_array, axis=0)

# Calculate confidence interval
ci_factor = 1.96 / np.sqrt(len(emresults_list))  # 95% CI for the mean
ci_results = ci_factor * std_results

output_t = scenario_properties.scen_times

os.makedirs('figures', exist_ok=True)

# Plotting species population per shell (with CI)
fig, axes = plt.subplots(1, n_species, figsize=(20, 6))

cmap = plt.cm.viridis
norm = plt.Normalize(vmin=0, vmax=num_shells - 1)
colors = [cmap(norm(sh)) for sh in range(num_shells)]

for species_index in range(n_species):
    ax = axes.flatten()[species_index]
    species_data_mean = mean_results[species_index * num_shells:(species_index + 1) * num_shells]
    species_data_ci = ci_results[species_index * num_shells:(species_index + 1) * num_shells]

    for shell_index in range(num_shells):
        mean_curve = species_data_mean[shell_index]
        ci_curve = species_data_ci[shell_index]
        color = colors[shell_index]

        ax.plot(output_t, mean_curve, color=color, label=f'Shell {shell_index + 1}')
        ax.fill_between(output_t, mean_curve - ci_curve, mean_curve + ci_curve, color=color, alpha=0.2)

    ax.set_title(f'{species_names[species_index]}')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

plt.suptitle('Species across All Shells with 95% CI')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('figures/species_all_shells_SDE.png')
plt.close(fig)

# Plotting Total Objects (with CI)
plt.figure(figsize=(10, 6))

total_objects_all_species_mean = np.zeros_like(output_t)
total_objects_all_species_ci = np.zeros_like(output_t)

for i in range(n_species):
    start_idx = i * num_shells
    end_idx = start_idx + num_shells

    total_mean = np.sum(mean_results[start_idx:end_idx, :], axis=0)
    total_ci = np.sqrt(np.sum(ci_results[start_idx:end_idx, :] ** 2, axis=0))  # CI propagation assuming independence

    plt.plot(output_t, total_mean, label=f'{species_names[i]}')
    plt.fill_between(output_t, total_mean - total_ci, total_mean + total_ci, alpha=0.2)

    total_objects_all_species_mean += total_mean
    total_objects_all_species_ci += total_ci ** 2  # Variances add up

total_objects_all_species_ci = np.sqrt(total_objects_all_species_ci)

plt.plot(output_t, total_objects_all_species_mean, label='Total All Species', color='k', linewidth=2, linestyle='--')
plt.fill_between(output_t,
                 total_objects_all_species_mean - total_objects_all_species_ci,
                 total_objects_all_species_mean + total_objects_all_species_ci,
                 color='gray', alpha=0.2)

plt.xlabel('Time')
plt.ylabel('Total Number of Objects')
plt.title('Objects Over Time with 95% Confidence Interval')
plt.xlim(0, 100)
#plt.ylim(0, 2500)
plt.legend()
plt.tight_layout()
plt.savefig('figures/total_objects_over_time_SDE.png')
plt.close()
